In [1]:
import pymysql
import pandas as pd
import numpy as np
from time import time as timeStamp
from random import random as rand
import os
import time


/opt/intel/intelpython3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
class DataBase:
    def __init__(self, host, user, pwd, port, database):
        self.host = host
        self.user = user
        self.pwd = pwd
        self.port = port
        self.database = database
        self.connection = pymysql.connect(host=self.host, user=self.user, passwd=self.pwd, port=self.port, database=self.database, charset='utf8mb4', use_unicode=True)
        self.connection.autocommit(True)
        self.pointer = self.connection.cursor()

    def check_connect(self):
        self.connection.ping(reconnect=True)
        self.pointer = self.connection.cursor()
        return

    def close_db(self):
        self.connection.close()
        self.pointer.close()
        return

    
    def query(self, sql, din = None):
        if din is not None:
            try:
                self.check_connect()
                rows = self.pointer.execute(sql, din)
                if rows > 0:
                    results = self.pointer.fetchall()
                else:
                    results = []
                return results
            except Exception as e:
                with open("./logs/query_error.log", 'a+') as f:
                    f.write(_ver_+"SQL: {}\nError: {}\n\n".format(sql, str(e)))

        else:
            try:
                self.check_connect()
                rows = self.pointer.execute(sql)
                if rows > 0:
                    results = self.pointer.fetchall()
                else:
                    results = []
                return results
            except Exception as e:
                with open("./logs/query_error.log", 'a+') as f:
                    f.write(_ver_+"SQL: {}\nError: {}\n\n".format(sql, str(e)))
                    
    def update(self, sql, din = None):
        rows = 0
        if din != None:
            try:
                self.check_connect()
                rows = self.pointer.execute(sql, din)
                if rows > 0:
                    self.connection.commit()
            except pymysql.err.IntegrityError:
                pass
            except Exception as e:
                self.check_connect()
                self.connection.rollback()
                self.connection.commit()
                with open("./logs/query_error.log", 'a+') as f:
                    f.write("SQL: {}\nError: {}\n\n".format(sql, str(e)))

        else:
            try:
                self.check_connect()
                rows = self.pointer.execute(sql)
                if rows > 0:
                    self.connection.commit()
            except pymysql.err.IntegrityError:
                pass
            except Exception as e:
                self.check_connect()
                self.connection.rollback()
                self.connection.commit()
                with open("./logs/query_error.log", 'a+') as f:
                    f.write("SQL: {}\nError: {}\n\n".format(sql, str(e)))

        return rows
           
    

In [3]:
# 此處已處理為假的資訊
db = DataBase(host='server_ip', user='user_name', pwd='user_password', port=port, database='project_database')

In [4]:
if not os.path.isfile("./gacha_items.csv"):
    gacha_items = []
    for x in db.query("SELECT item_name, item_limit FROM `gacha_items`;"):
        gacha_items.append(list(x))
    gacha_items = pd.DataFrame(gacha_items, columns=['NAME', 'limit'])
    gacha_items['value'] = gacha_items.NAME.apply(lambda x:int(x.split('元')[0].split('〉')[-1].split('店')[-1]))
    
    gacha_items = gacha_items.sort_values(by='value', ascending=False).reset_index(drop=True)
    gacha_items.to_csv("./gacha_items.csv", index=False)
else:
    gacha_items = pd.read_csv("./gacha_items.csv")
    



gacha_items

,NAME,limit,value
0,〈隨心選〉3000元多選一即享券,1,3000
1,〈隨心選〉2000元多選一即享券,2,2000
2,〈隨心選〉1000元多選一即享券,3,1000
3,〈隨心選〉便利商店100元購物金三選一即享券,14,100


In [5]:
if not os.path.isfile("./gacha_user.csv"):
    users = []
    for x in db.query("SELECT fake_email, seeds, prob FROM `gacha_user`;"):
        users.append(list(x))
    users = pd.DataFrame(users, columns=['NAME', 'seeds', 'prob'])
    users.seeds = users.seeds.astype('int')
    users.to_csv("./gacha_user.csv", index=False)
else:
    users = pd.read_csv("./gacha_user.csv")


users

,NAME,seeds,prob
0,q1****23@gmail.com,4592,0.951101
1,yan0*****7698@gmail.com,3850,0.797417
2,boch*****en1123@gmail.com,3702,0.766763
3,sin***407@gmail.com,4899,1.014690
4,cut***2014@gmail.com,4247,0.879644
5,amo***913@gmail.com,3402,0.704626
6,mo****ng@gmail.com,5926,1.227400
7,jj****17@gmail.com,3390,0.702141
8,x0****63@yahoo.com.tw,4349,0.900770
9,er****52@gmail.com,4345,0.899942


In [6]:
def GACHA():
    if not os.path.isdir("./gacha_history"):
        os.mkdir("./gacha_history")

    if os.path.isfile("./抽獎結果.csv"):
        print("抽獎已完成！重現抽獎結果！\n")
        result = pd.read_csv("./抽獎結果.csv")

        results = []
        for idx in gacha_items.index:
            print("====================================================================")
            print("正在抽出第 {}個獎項: 『{}』，本獎項有 {}份".format(idx+1, gacha_items.loc[idx, 'NAME'], gacha_items.loc[idx, 'limit']))
            for item_cnt in range(gacha_items.loc[idx, 'limit']):
                ts =  result.loc[idx, 'ts']
                state = int(str(ts).split('.')[-1])
                print("正在抽出第 {}份...隨機數：{}".format(item_cnt+1, state))
                np.random.seed(seed=state)

                takarakujis = []
                for u_idx in users.index:
                    np.random.shuffle(takarakujis)
                    for seed in range(users.loc[u_idx, 'seeds']):
                        takarakujis.append(users.loc[u_idx, 'NAME'])

                np.random.shuffle(takarakujis)
                slt = np.random.randint(len(takarakujis))

                print("得獎者代號：{}，原始籤數：{}，原始中獎率：{:.4}%".format(takarakujis[slt], users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0], users.loc[users.NAME == takarakujis[slt], 'prob'].values[0]))
                results.append([
                    ts,
                    state,
                    slt,
                    takarakujis[slt], 
                    users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0],
                    gacha_items.loc[idx, 'value'],
                    users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0] - gacha_items.loc[idx, 'value'], 
                    gacha_items.loc[idx, 'NAME']
                ])
                users.loc[users.NAME == takarakujis[slt], 'seeds'] = max(users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0] - gacha_items.loc[idx, 'value'], 0)
                users['prob'] = users['seeds'] / np.sum(users['seeds']) * 100
                print("得獎者剩餘籤數：{}，剩餘機率：{:.4}%\n".format(users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0], users.loc[users.NAME == takarakujis[slt], 'prob'].values[0]))

    else:
        results = []
        for idx in gacha_items.index:
            print("====================================================================")
            print("正在抽出第 {}個獎項: 『{}』，本獎項有 {}份".format(idx+1, gacha_items.loc[idx, 'NAME'], gacha_items.loc[idx, 'limit']))
            for item_cnt in range(gacha_items.loc[idx, 'limit']):

                ts = timeStamp()
                state = int(str(ts).split('.')[-1])
                print("正在抽出第 {}份...隨機數：{}".format(item_cnt+1, state))
                
                np.random.seed(seed=state)

                takarakujis = []

                for u_idx in users.index:
                    np.random.shuffle(takarakujis)

                    for seed in range(users.loc[u_idx, 'seeds']):
                        takarakujis.append(users.loc[u_idx, 'NAME'])

                    np.random.shuffle(takarakujis)

                np.random.shuffle(takarakujis)
                slt = np.random.randint(len(takarakujis))
                
                print("得獎者代號：{}，原始籤數：{}，原始中獎率：{:.4}%".format(takarakujis[slt], users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0], users.loc[users.NAME == takarakujis[slt], 'prob'].values[0]))
                results.append([
                    ts,
                    state,
                    slt,
                    takarakujis[slt], 
                    users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0],
                    gacha_items.loc[idx, 'value'],
                    users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0] - gacha_items.loc[idx, 'value'], 
                    gacha_items.loc[idx, 'NAME'],
                    state,
                    ts
                ])
                users.loc[users.NAME == takarakujis[slt], 'seeds'] = users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0] - gacha_items.loc[idx, 'value']
                users['prob'] = users['seeds'] / np.sum(users['seeds']) * 100
                print("得獎者剩餘籤數：{}，剩餘機率：{:.4}%\n".format(users.loc[users.NAME == takarakujis[slt], 'seeds'].values[0], users.loc[users.NAME == takarakujis[slt], 'prob'].values[0]))
                users.to_csv("./gacha_history/gacha_user_history_抽出{}份{}後.csv".format(item_cnt+1, gacha_items.loc[idx, 'NAME']), index=False)
                time.sleep(rand() * 3600) # 避免迴圈太快導致隨機數失效

    print("完成抽獎")
    result = pd.DataFrame(results, columns=['時間戳', '隨機數', '抽選位置', '得獎者代號', '原始籤數', '扣除籤數', '剩餘籤數', '獎項', 'state', 'ts'])
    result['時間戳'] = pd.to_datetime(result['時間戳'].astype('int64'), unit='s').apply(lambda x:x.tz_localize('UTC').tz_convert('Asia/Taipei'))
    result.to_csv("抽獎結果.csv", index=False)
    return result


In [7]:
result = GACHA()

正在抽出第 1個獎項: 『〈隨心選〉3000元多選一即享券』，本獎項有 1份
正在抽出第 1份...隨機數：5606892
得獎者代號：gil***705@gmail.com，原始籤數：4975，原始中獎率：1.03%
得獎者剩餘籤數：1975，剩餘機率：0.4116%

正在抽出第 2個獎項: 『〈隨心選〉2000元多選一即享券』，本獎項有 2份
正在抽出第 1份...隨機數：3327458
得獎者代號：alic*****iris@gmail.com，原始籤數：5301，原始中獎率：1.105%
得獎者剩餘籤數：3301，剩餘機率：0.6909%

正在抽出第 2份...隨機數：1730027
得獎者代號：kar******99@gmail.com，原始籤數：3332，原始中獎率：0.6973%
得獎者剩餘籤數：1332，剩餘機率：0.2799%

正在抽出第 3個獎項: 『〈隨心選〉1000元多選一即享券』，本獎項有 3份
正在抽出第 1份...隨機數：2492285
得獎者代號：wis***124@icloud.com，原始籤數：3732，原始中獎率：0.7843%
得獎者剩餘籤數：2732，剩餘機率：0.5754%

正在抽出第 2份...隨機數：3877573
得獎者代號：e077*****89322@gmail.com，原始籤數：3934，原始中獎率：0.8285%
得獎者剩餘籤數：2934，剩餘機率：0.6192%

正在抽出第 3份...隨機數：422442
得獎者代號：andy********0211@gmail.com，原始籤數：3846，原始中獎率：0.8117%
得獎者剩餘籤數：2846，剩餘機率：0.6019%

正在抽出第 4個獎項: 『〈隨心選〉便利商店100元購物金三選一即享券』，本獎項有 14份
正在抽出第 1份...隨機數：9313745
得獎者代號：ang******11@gmail.com，原始籤數：5367，原始中獎率：1.135%
得獎者剩餘籤數：5267，剩餘機率：1.114%

正在抽出第 2份...隨機數：4749749
得獎者代號：s0****22@gm.ncue.edu.tw，原始籤數：5102，原始中獎率：1.079%
得獎者剩餘籤數：5002，剩餘機率：1.058%

正在抽出第 3份...隨機數：48784

In [8]:
result

,時間戳,隨機數,抽選位置,得獎者代號,原始籤數,扣除籤數,剩餘籤數,獎項,state,ts
0,2019-06-16 13:16:31+08:00,5606892,391549,gil***705@gmail.com,4975,3000,1975,〈隨心選〉3000元多選一即享券,5606892,1.560662e+09
1,2019-06-16 13:20:01+08:00,3327458,168135,alic*****iris@gmail.com,5301,2000,3301,〈隨心選〉2000元多選一即享券,3327458,1.560662e+09
2,2019-06-16 13:53:44+08:00,1730027,155985,kar******99@gmail.com,3332,2000,1332,〈隨心選〉2000元多選一即享券,1730027,1.560664e+09
3,2019-06-16 13:56:40+08:00,2492285,202124,wis***124@icloud.com,3732,1000,2732,〈隨心選〉1000元多選一即享券,2492285,1.560665e+09
4,2019-06-16 13:56:56+08:00,3877573,438118,e077*****89322@gmail.com,3934,1000,2934,〈隨心選〉1000元多選一即享券,3877573,1.560665e+09
5,2019-06-16 14:00:36+08:00,422442,374210,andy********0211@gmail.com,3846,1000,2846,〈隨心選〉1000元多選一即享券,422442,1.560665e+09
6,2019-06-16 14:01:07+08:00,9313745,373072,ang******11@gmail.com,5367,100,5267,〈隨心選〉便利商店100元購物金三選一即享券,9313745,1.560665e+09
7,2019-06-16 14:02:43+08:00,4749749,212901,s0****22@gm.ncue.edu.tw,5102,100,5002,〈隨心選〉便利商店100元購物金三選一即享券,4749749,1.560665e+09
8,2019-06-16 14:04:17+08:00,4878454,443799,q1****23@gmail.com,4592,100,4492,〈隨心選〉便利商店100元購物金三選一即享券,4878454,1.560665e+09
9,2019-06-16 14:06:23+08:00,6770592,436934,s0****26@gm.ncue.edu.tw,5350,100,5250,〈隨心選〉便利商店100元購物金三選一即享券,6770592,1.560665e+09
